In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
feature_list = ['x','y','minute_sin','minute_cos','w0','w1','w2','w3','w4','w5','w6','day','month','year','accuracy']
#for i in range(0,24):
#    feature_list.append('h' + str(i))

In [3]:
def prepare_data(df):    
    #Feature engineering
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-02T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    #df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    #for i in range(0,24):
    #    df['h' + str(i)] = (((d_times.hour+ d_times.minute/60) + i) % 24)# * fw[2]
    minute = 2*np.pi*((df["time"]//5)%288)/288
    df['minute_sin'] = (np.sin(minute)+1).round(4) * fw[2]
    df['minute_cos'] = (np.cos(minute)+1).round(4) * fw[2]
    del minute
    
    df['w0'] = ((d_times.weekday + 0) % 7) * fw[3]
    df['w1'] = ((d_times.weekday + 1) % 7) * fw[3]
    df['w2'] = ((d_times.weekday + 2) % 7) * fw[3]
    df['w3'] = ((d_times.weekday + 3) % 7) * fw[3]
    df['w4'] = ((d_times.weekday + 4) % 7) * fw[3]
    df['w5'] = ((d_times.weekday + 5) % 7) * fw[3]
    df['w6'] = ((d_times.weekday + 6) % 7) * fw[3]
    
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    #df.accuracy = df.accuracy.values * fw[7]
    df['accuracy'] = np.log10(df.accuracy) * fw[7]
    df['log_month'] = np.log10(3+df.time/(60 * 24 * 30)) * fw[8]
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def calculate_distance(distances):
    return distances ** -2

In [5]:
def process_one_cell(df_train, df_test, th):    
    place_counts = df_train.place_id.value_counts()
    mask = (place_counts[df_train.place_id.values] >= th).values
    df_train = df_train.loc[mask]
    row_ids = df_test.row_id
    
    best_k=np.floor(np.sqrt(len(df_train.index)/4*best_mul)/5)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=best_k.astype(int), weights=calculate_distance, 
                               metric='manhattan')
    clf.fit(df_train[feature_list], df_train.place_id)
    predictions = clf.predict_proba(df_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [6]:
def run_prediction(df_train, df_test):
    df_train = prepare_data(df_train)
    df_test = prepare_data(df_test)
    df_test['p1'] = np.nan
    df_test['p2'] = np.nan
    df_test['p3'] = np.nan
    
    prediction_result = process_one_cell(df_train, df_test, 1)
    prediction_result.sort_index(inplace=True)
    return prediction_result

In [7]:
def run_validation():
    df = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.time <= 786239 * 0.875) & \
                               (df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_validation_cell = df[(df.time > 786239 * 0.875) & \
                                    (df.x >= min_x) & \
                                    (df.x < max_x) & \
                                    (df.y >= min_y) & \
                                    (df.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_validation_cell)
            
            # Calculate score
            prediction_result.sort_index(inplace=True)
            prediction_result['score'] = (prediction_result.p1 == df_validation_cell.place_id) * 1
            prediction_result['score'] += (prediction_result.p2 == df_validation_cell.place_id) * 0.5
            prediction_result['score'] += (prediction_result.p3 == df_validation_cell.place_id) * 0.33

            score = prediction_result.score.mean()
            #print('s1:', score, flush=True)
            total_score += score
            score_count += 1
            
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
    print("Final:", total_score/score_count, flush=True)

In [8]:
fw = [400, 1000, 0.5, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 1, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 2, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 3, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 4, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

Elapsed time overall: 258.0870931148529 seconds 0
Elapsed time overall: 287.4778823852539 seconds 1
Elapsed time overall: 326.93365716934204 seconds 2
Elapsed time overall: 314.255984544754 seconds 3
Elapsed time overall: 308.0640227794647 seconds 4
Elapsed time overall: 321.4301965236664 seconds 5
Elapsed time overall: 308.742390871048 seconds 6
Elapsed time overall: 321.76100063323975 seconds 7
Elapsed time overall: 296.36202335357666 seconds 8
Elapsed time overall: 217.0418577194214 seconds 9
Final: 0.501140441115359
Elapsed time overall: 219.89807868003845 seconds 0
Elapsed time overall: 241.37751650810242 seconds 1
Elapsed time overall: 266.9363160133362 seconds 2
Elapsed time overall: 237.4693365097046 seconds 3
Elapsed time overall: 229.33096933364868 seconds 4
Elapsed time overall: 250.3716950416565 seconds 5
Elapsed time overall: 238.13357019424438 seconds 6
Elapsed time overall: 240.57806158065796 seconds 7
Elapsed time overall: 224.45972228050232 seconds 8
Elapsed time overa

In [9]:
fw = [400, 1000, 10, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

Elapsed time overall: 250.96529960632324 seconds 0
Elapsed time overall: 265.2607419490814 seconds 1
Elapsed time overall: 299.222243309021 seconds 2
Elapsed time overall: 283.1967840194702 seconds 3
Elapsed time overall: 289.15671396255493 seconds 4
Elapsed time overall: 288.6541197299957 seconds 5
Elapsed time overall: 275.2561459541321 seconds 6
Elapsed time overall: 294.1309492588043 seconds 7
Elapsed time overall: 273.03186297416687 seconds 8
Elapsed time overall: 236.32695722579956 seconds 9
Final: 0.5478232270120753


In [10]:
fw = [400, 1000, 8, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 9, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 11, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 12, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 13, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

fw = [400, 1000, 14, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

Elapsed time overall: 202.21201467514038 seconds 0
Elapsed time overall: 247.13805079460144 seconds 1
Elapsed time overall: 304.08170676231384 seconds 2
Elapsed time overall: 257.74796319007874 seconds 3
Elapsed time overall: 248.43470740318298 seconds 4
Elapsed time overall: 256.6589984893799 seconds 5
Elapsed time overall: 246.9948227405548 seconds 6
Elapsed time overall: 241.57336473464966 seconds 7
Elapsed time overall: 221.2356424331665 seconds 8
Elapsed time overall: 189.88158559799194 seconds 9
Final: 0.5461719398724764
Elapsed time overall: 196.79955291748047 seconds 0
Elapsed time overall: 217.74723863601685 seconds 1
Elapsed time overall: 241.6849193572998 seconds 2
Elapsed time overall: 229.93262839317322 seconds 3
Elapsed time overall: 231.6329083442688 seconds 4
Elapsed time overall: 239.34436869621277 seconds 5
Elapsed time overall: 228.76018977165222 seconds 6
Elapsed time overall: 239.63643217086792 seconds 7
Elapsed time overall: 222.8177011013031 seconds 8
Elapsed tim

	Line 11: Final: 0.5461719398724764
	Line 22: Final: 0.5472564355049623
	Line 33: Final: 0.5480990281717935
	Line 44: Final: 0.5480826137006548
	Line 55: Final: 0.5478676239016081
	Line 66: Final: 0.5475174382668965

In [13]:
def run_test():
    
    # Run test
    df = pd.read_csv('../../train.csv',
                           usecols=['row_id','x','y','accuracy','time','place_id'])
    df_test = pd.read_csv('../../test.csv',
                           usecols=['row_id','x','y','accuracy','time'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    total_result = pd.DataFrame()
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_test_cell = df_test[(df_test.x >= min_x) & \
                                   (df_test.x < max_x) & \
                                   (df_test.y >= min_y) & \
                                   (df_test.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_test_cell)
            total_result = total_result.append(prediction_result)
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
            
    
    total_result.sort_index(inplace=True)
    total_result['place_id'] = total_result.p1.astype(str) + " " + \
                               total_result.p2.astype(str) + " " + \
                               total_result.p3.astype(str)
    total_result[['row_id', 'place_id']].to_csv('Baseline620-WithSin.csv', index=False)

In [14]:
fw = [400, 1000, 11, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_test()

Elapsed time overall: 526.0489730834961 seconds 0
Elapsed time overall: 617.4664888381958 seconds 1
Elapsed time overall: 610.2331659793854 seconds 2
Elapsed time overall: 579.0467178821564 seconds 3
Elapsed time overall: 578.9334514141083 seconds 4
Elapsed time overall: 599.3313109874725 seconds 5
Elapsed time overall: 476.83973264694214 seconds 6
Elapsed time overall: 495.81134128570557 seconds 7
Elapsed time overall: 468.79841804504395 seconds 8
Elapsed time overall: 420.8868474960327 seconds 9
